# How to extract data from HF repository

The idea is to get familiarize with the HF API to extract data from ML repositories.
Is important to realize that relevant metadata basically can be found in the readme.MD file.

### Download a specific file

In the quickstart guide of the [Hugging Face Hub API](https://huggingface.co/docs/huggingface_hub/v0.8.0/en/quick-start), there is an example on how to retrieve particular documents from the ML repository.  

In [12]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="google/pegasus-xsum", filename="README.md")

'/home/vscode/.cache/huggingface/hub/models--google--pegasus-xsum/snapshots/8d8ffc158a3bee9fbb03afacdfc347c823c5ec8b/README.md'

Now, we would like to pick where is the document being downloaded. Even check if we can bring it to memory right away.

For the first problem we have the following [documentation](https://huggingface.co/docs/huggingface_hub/main/en/guides/download). We can use the `local_dir="path/to/folder"` attribute of the `hf_hub_download` function.

In [13]:
hf_hub_download(repo_id="google/pegasus-xsum", filename="README.md",local_dir="/home/vscode/README_files")

'/home/vscode/README_files/README.md'

### Model cards
Even better than the above, there are ways to only query the Model card metadata that is usually on the README.md file, using the [ModelCard](https://huggingface.co/docs/huggingface_hub/main/en/guides/model-cards) function. 

Now the problem with this approach is clear, not all the metadata can be found on the Model card, a lot of useful information can be extracted from the README.md file.

Even with the previous setback in can said that this functionality really helps to proccess the data better. 

Now I don't know if it is worth to spend one of the 10k limited requests we have every minute on a preprocessing step. 

In [15]:
from huggingface_hub import ModelCard

card = ModelCard.load('google/pegasus-xsum')

#Get the textual part of the README.md file of the model 
print(card.text)

#Get the metadata part of README.md file of the model 
print(card.data)


### Pegasus Models
See Docs: [here](https://huggingface.co/transformers/master/model_doc/pegasus.html)

Original TF 1 code [here](https://github.com/google-research/pegasus)

Authors: Jingqing Zhang, Yao Zhao, Mohammad Saleh and Peter J. Liu on Dec 18, 2019

Maintained by: [@sshleifer](https://twitter.com/sam_shleifer)

Task: Summarization

The following is copied from the authors' README.

# Mixed & Stochastic Checkpoints

We train a pegasus model with sampled gap sentence ratios on both C4 and HugeNews, and stochastically sample important sentences. The updated the results are reported in this table.

| dataset | C4 | HugeNews | Mixed & Stochastic|
| ---- | ---- | ---- | ----|
| xsum | 45.20/22.06/36.99 | 47.21/24.56/39.25 | 47.60/24.83/39.64|
| cnn_dailymail | 43.90/21.20/40.76 | 44.17/21.47/41.11 | 44.16/21.56/41.30|
| newsroom | 45.07/33.39/41.28 | 45.15/33.51/41.33 | 45.98/34.20/42.18|
| multi_news | 46.74/17.95/24.26 | 47.52/18.72/24.91 | 47.65/18.75/24.95|
| gigaword | 38.75/1

` card.data ` : Returns a ModelCardData instance with the model card’s metadata. 
Call `.to_dict()` on this instance to get the representation as a dictionary.

In [4]:
print(card.data.to_dict())

{'language': 'en', 'tags': ['summarization'], 'model-index': [{'name': 'google/pegasus-xsum', 'results': [{'task': {'type': 'summarization', 'name': 'Summarization'}, 'dataset': {'name': 'samsum', 'type': 'samsum', 'config': 'samsum', 'split': 'train'}, 'metrics': [{'type': 'rouge', 'value': 21.8096, 'name': 'ROUGE-1', 'verified': True}, {'type': 'rouge', 'value': 4.2525, 'name': 'ROUGE-2', 'verified': True}, {'type': 'rouge', 'value': 17.4469, 'name': 'ROUGE-L', 'verified': True}, {'type': 'rouge', 'value': 18.8907, 'name': 'ROUGE-LSUM', 'verified': True}, {'type': 'loss', 'value': 3.0317161083221436, 'name': 'loss', 'verified': True}, {'type': 'gen_len', 'value': 20.3122, 'name': 'gen_len', 'verified': True}]}, {'task': {'type': 'summarization', 'name': 'Summarization'}, 'dataset': {'name': 'xsum', 'type': 'xsum', 'config': 'default', 'split': 'test'}, 'metrics': [{'type': 'rouge', 'value': 46.8623, 'name': 'ROUGE-1', 'verified': True}, {'type': 'rouge', 'value': 24.4533, 'name': 'RO

### List models

Now the next step is how to get the names of the model repositories. After that, it would be nice to figure out a way to get the names of models based on a time period.

Let's explore the [list_models](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/hf_api#huggingface_hub.HfApi.list_models) function.

We can access relevant information of the model by using the properties of the [ModelInfo](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/hf_api#huggingface_hub.hf_api.ModelInfo) Object.

In [6]:
from huggingface_hub import HfApi

api = HfApi()

# List all models
model_list = api.list_models()

i = 5

for model in model_list:
    if(i==0):
        break
    i-=1
    print(model.id ,model.created_at)

albert/albert-base-v1 2022-03-02 23:29:04+00:00
albert/albert-base-v2 2022-03-02 23:29:04+00:00
albert/albert-large-v1 2022-03-02 23:29:04+00:00
albert/albert-large-v2 2022-03-02 23:29:04+00:00
albert/albert-xlarge-v1 2022-03-02 23:29:04+00:00


#### Order in the lists

After around 30000 of the initial models, different dates start to appear, and they look to be orderd in an accedent pattern.
The initial date appears to be 2022-03-02 23:29:04+00:00, and then another batch with 2022-03-02 23:29:05+00:00 appears as well.
Retreiving the general information from each model seems to be fast.

In [30]:
model_list = api.list_models()
i = 30000
unique_dates = set()
for model in model_list:
    if(i==0):
        break
    i-=1
    unique_dates.add(model.created_at)

print(unique_dates)
print(len(unique_dates))

{datetime.datetime(2022, 3, 2, 23, 29, 4, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 3, 2, 23, 29, 5, tzinfo=datetime.timezone.utc)}
2


In [35]:
model_list = api.list_models()
i = 31000
unique_dates = set()
for model in model_list:
    if(i==0):
        break
    i-=1
    unique_dates.add(model.created_at)

print(len(unique_dates))
last_10_dates = list(unique_dates)[-10:]
print([str(x) for x in last_10_dates])

892
['2022-03-09 11:52:08+00:00', '2022-03-09 13:30:22+00:00', '2022-03-03 15:53:14+00:00', '2022-03-04 12:49:07+00:00', '2022-03-07 00:22:21+00:00', '2022-03-08 14:22:06+00:00', '2022-03-03 04:53:15+00:00', '2022-03-06 20:35:13+00:00', '2022-03-05 20:52:57+00:00', '2022-03-07 09:51:17+00:00']


### Combine the methods

We can apply the knowledge of the model cards to the list of model information we got in the last section.

The `ModelInfo` object returned by `list_models()` has the following restrictions:

    Most attributes of this class are optional. This is because the data returned by the Hub depends on the query made. In general, the more specific the query, the more information is returned. On the contrary, when listing models using list_models() only a subset of the attributes are returned.

The above means that, the data from the model card is not brought when the `list_models()` method is called.

In [11]:
model_list = api.list_models()
i = 3
found_metadata_list = []

for model in model_list:
    if(i==0):
        break
    if(model.card_data!=None):
        found_metadata_list.append(model)
        print(model.id)
        print(model.card_data)
        i-=1

The solution is to use the ModelCard.load() function to bring the metadata information.
Here an error occurs where, if the model does not have a model card, the function throws the following:

    EntryNotFoundError: 404 Client Error. (Request ID: Root=1-65bab81f-55f2b8de5181a4be4982085b;0edd9b55-c840-4d40-a05e-d03930f3addf)

    Entry Not Found for url: https://huggingface.co/zhuqing/bert-base-uncased-mumsnet-all-0/resolve/main/README.md.

You need to import the error from the HF utils libraries and handle it.

In [31]:
from huggingface_hub.utils import EntryNotFoundError

model_list = api.list_models()
i = 3
start = 30000
found_metadata_list = []

for model in model_list:
    if(start>0):
        start-=1
        continue
    
    if(i==0):
        break
    
    model_card_data = None
    try:
        model_card_data = ModelCard.load(model.id)
    except EntryNotFoundError:
        print(f"Error loading model card")
    
    if(model_card_data!=None):
        print()
        print(model.id)
        print(card.data.to_dict())
        i-=1

Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card
Error loading model card


README.md: 100%|██████████| 1.83k/1.83k [00:00<00:00, 6.28MB/s]
/home/vscode/.local/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")



ziedsb19/tunbert_zied
{'language': 'en', 'tags': ['summarization'], 'model-index': [{'name': 'google/pegasus-xsum', 'results': [{'task': {'type': 'summarization', 'name': 'Summarization'}, 'dataset': {'name': 'samsum', 'type': 'samsum', 'config': 'samsum', 'split': 'train'}, 'metrics': [{'type': 'rouge', 'value': 21.8096, 'name': 'ROUGE-1', 'verified': True}, {'type': 'rouge', 'value': 4.2525, 'name': 'ROUGE-2', 'verified': True}, {'type': 'rouge', 'value': 17.4469, 'name': 'ROUGE-L', 'verified': True}, {'type': 'rouge', 'value': 18.8907, 'name': 'ROUGE-LSUM', 'verified': True}, {'type': 'loss', 'value': 3.0317161083221436, 'name': 'loss', 'verified': True}, {'type': 'gen_len', 'value': 20.3122, 'name': 'gen_len', 'verified': True}]}, {'task': {'type': 'summarization', 'name': 'Summarization'}, 'dataset': {'name': 'xsum', 'type': 'xsum', 'config': 'default', 'split': 'test'}, 'metrics': [{'type': 'rouge', 'value': 46.8623, 'name': 'ROUGE-1', 'verified': True}, {'type': 'rouge', 'value

README.md: 100%|██████████| 57.0/57.0 [00:00<00:00, 191kB/s]



zinary/DialoGPT-small-rick-new
{'language': 'en', 'tags': ['summarization'], 'model-index': [{'name': 'google/pegasus-xsum', 'results': [{'task': {'type': 'summarization', 'name': 'Summarization'}, 'dataset': {'name': 'samsum', 'type': 'samsum', 'config': 'samsum', 'split': 'train'}, 'metrics': [{'type': 'rouge', 'value': 21.8096, 'name': 'ROUGE-1', 'verified': True}, {'type': 'rouge', 'value': 4.2525, 'name': 'ROUGE-2', 'verified': True}, {'type': 'rouge', 'value': 17.4469, 'name': 'ROUGE-L', 'verified': True}, {'type': 'rouge', 'value': 18.8907, 'name': 'ROUGE-LSUM', 'verified': True}, {'type': 'loss', 'value': 3.0317161083221436, 'name': 'loss', 'verified': True}, {'type': 'gen_len', 'value': 20.3122, 'name': 'gen_len', 'verified': True}]}, {'task': {'type': 'summarization', 'name': 'Summarization'}, 'dataset': {'name': 'xsum', 'type': 'xsum', 'config': 'default', 'split': 'test'}, 'metrics': [{'type': 'rouge', 'value': 46.8623, 'name': 'ROUGE-1', 'verified': True}, {'type': 'rouge

README.md: 100%|██████████| 31.0/31.0 [00:00<00:00, 73.0kB/s]


ziqingyang/XLMRobertaBaseForXNLI-en
{'language': 'en', 'tags': ['summarization'], 'model-index': [{'name': 'google/pegasus-xsum', 'results': [{'task': {'type': 'summarization', 'name': 'Summarization'}, 'dataset': {'name': 'samsum', 'type': 'samsum', 'config': 'samsum', 'split': 'train'}, 'metrics': [{'type': 'rouge', 'value': 21.8096, 'name': 'ROUGE-1', 'verified': True}, {'type': 'rouge', 'value': 4.2525, 'name': 'ROUGE-2', 'verified': True}, {'type': 'rouge', 'value': 17.4469, 'name': 'ROUGE-L', 'verified': True}, {'type': 'rouge', 'value': 18.8907, 'name': 'ROUGE-LSUM', 'verified': True}, {'type': 'loss', 'value': 3.0317161083221436, 'name': 'loss', 'verified': True}, {'type': 'gen_len', 'value': 20.3122, 'name': 'gen_len', 'verified': True}]}, {'task': {'type': 'summarization', 'name': 'Summarization'}, 'dataset': {'name': 'xsum', 'type': 'xsum', 'config': 'default', 'split': 'test'}, 'metrics': [{'type': 'rouge', 'value': 46.8623, 'name': 'ROUGE-1', 'verified': True}, {'type': '

In [ ]:
from huggingface_hub import EntryNotFoundError

model_list = api.list_models()
i = 3
start = 30000
found_metadata_list = []

for model in model_list:
    if(start>0):
        start-=1
        continue
    
    if(i==0):
        break
    
    model_card_data = None
    try:
        model_card_data = ModelCard.load(model.id)
    except HTTPError:
        print(f"Error loading model card")
    
    if(model_card_data!=None):
        print()
        print(model.id)
        print(card.data.to_dict())
        i-=1

ImportError: cannot import name 'EntryNotFoundError' from 'huggingface_hub' (/home/vscode/.local/lib/python3.11/site-packages/huggingface_hub/__init__.py)

### Faster downloads
If you are running on a machine with high bandwidth, you can increase your download speed with [hf_transfer](https://github.com/huggingface/hf_transfer), a Rust-based library developed to speed up file transfers with the Hub. To enable it:

- Specify the hf_transfer extra when installing huggingface_hub (e.g. pip install huggingface_hub[hf_transfer]).
- Set HF_HUB_ENABLE_HF_TRANSFER=1 as an environment variable.

Test will have to be made to actually determine how worth it it is to use this feature, given that the README.md files weight between 3KB to 10KB.